<a id=’Business_Understanding’></a>
## Business Understanding

We have been tasked with developing a machine learning model for predicting the functionality of a water point (i.e., well) in Tanzania. Our data set was given to us from an unnamed source, and the aim of the project is unclear to us:

<a id=’Installation_and_Imports’></a>
### Installation and Imports

Before getting to work with the data, we begin by first importing all of the packages required throughout the notebook. This is done at the beginning of the notebook so that we can keep track of what packages we are using, as well as what variables we have assigned to the packages for method calls. 

In [ ]:
# !pip install xgboost
# !pip install lazypredict
# !pip install imblearn
# !pip install sklearn --upgrade

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statistics
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import uniform, truncnorm, randint
from sklearn.model_selection import RandomizedSearchCV
#from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from IPython.display import Image

<a id=’Data_Understanding’></a>
## Data Understanding

Before we analyse and model our data, we must first understand what exactly it is that our data represents. This information we have collected in a Data Dictionary. Having completed this step we will then seek to develop an idea of the basic shape and properties of the data set. 

### Data dictionary

amount_tsh - Total static head (amount water available to waterpoint)

date_recorded - The date the row was entered

funder - Who funded the well

gps_height - Altitude of the well

installer - Organization that installed the well

longitude - GPS coordinate

latitude - GPS coordinate

wpt_name - Name of the waterpoint if there is one

num_private -

basin - Geographic water basin

subvillage - Geographic location

region - Geographic location

region_code - Geographic location (coded)

district_code - Geographic location (coded)

lga - Geographic location

ward - Geographic location

population - Population around the well

public_meeting - True/False

recorded_by - Group entering this row of data

scheme_management - Who operates the waterpoint

scheme_name - Who operates the waterpoint

permit - If the waterpoint is permitted

construction_year - Year the waterpoint was constructed

extraction_type - The kind of extraction the waterpoint uses

extraction_type_group - The kind of extraction the waterpoint uses

extraction_type_class - The kind of extraction the waterpoint uses

management - How the waterpoint is managed

management_group - How the waterpoint is managed

payment - What the water costs

payment_type - What the water costs

water_quality - The quality of the water

quality_group - The quality of the water

quantity - The quantity of water

quantity_group - The quantity of water

source - The source of the water

source_type - The source of the water

source_class - The source of the water

waterpoint_type - The kind of waterpoint

waterpoint_type_group - The kind of waterpoint

### Data Shape and Properties

In [ ]:
pump_train = pd.read_csv('pump_train.csv')
pump_test = pd.read_csv('pump_test.csv')

In [ ]:
pump_train.shape

In [ ]:
pump_test.shape

Our training data consists of 50490 data entries with attributes spread over 41 variables. 

Our training data consists of 8910 data entries with attributes spread over 41 variables. 

In [ ]:
pump_train.head().T

In [ ]:
pump_train.info()

The data types for our variables span from numerical values (int64 and float64) to what appears to be categorical values (object). 

In [ ]:
pump_train.isnull().sum()

In [ ]:
pump_test.isnull().sum()

Unfortunately, we have a lot of null values in our data set. This will need to be taken into consideration in the data analysis and eventually rectified in data preparation.

In [ ]:
pump_train.nunique()

### Data shape and properties summary

We have been tasked with developing a machine learning model to predict the value of the variable 'status_group', which variable can carry three possible values: 'non_functional', 'functional needs repair', and 'functional'. Our training data set is composed of 50490 observations cast over 41 variables. This means that we have 40 features that we can take into consideration when training our model. Whether or not it makes sense to include all of these features in our training is a matter to be considered later the data analysis and data preparation phases. 

Unfortunately, not all of our observations are complete. Indeed, 7 out of our 40 features have missing values. This will need to be taken into consideration during the data visualisation phase, and then later rectified in the data preparation phase. 

Also important to take note of here are the data types of the various features. More than half of our features carry the data type 'object'. This is problematic, as many machine learning models have difficulties with non-numerical values. This will also need to be addressed during the data preparation phase. 

Finally, those features which are expressed numerically, either as an int value or a float value, are scaled differently. If these values are feed directly into a training algorithim then they could lead to skewed results, and as such, this must also be addressed during the data preparation phase.   

We proceed now to data analysis.

<a id=’Data_Analysis’></a>

# Data Analysis

The following section is broken into the following three sub-sections: numerical features, categorical features, and target variable.

A mixture of statistical analyses and data visualisations should provide us with a better understanding of the data set that we are working with. This will, moreover, provide us with a set of action points for the data preparation phase, as well as important insights for the modelling phase. 

## Numerical Features

Let us now take a closer look at our attributes with numerical values, i.e., int or float data types. 
First we will call up some basic statistics.

In [ ]:
pump_train.describe()

In [ ]:
pump_train.hist(bins=50, density=False, figsize=(12,10))  # density=False would make counts

Id obviously tells us nothing about any individual water point and we can accordingly disregard it for the purposes of data analysis. 

The attribute 'amount_tsh' (amount of water available at water point) is, however, worthy of further consideration. From the statistical analysis we know that many of the water points have no water in them. Whether or not these observations are veridical is questionable. 

The 'gps_height' attribute refers to the altitude above sea level of the water point. This is also relevant to our analyses. The histogramm shows that the vast majority of water points are clustered around sea level, but there is also no shortage of water points located at higher altitudes up to the highest at 2770 metres above sea level. 

For the attributes 'longitude' and 'latitude' we require more advanced visualisations that place water points on a map for them to be of more assistance. We will leave this to later. 

We do not have a data dictionary entry for the attribute 'num_private' and we will accordingly leave this attribute out of our analysis. 

The attributes 'region code' and 'district code' require further information to be interpreted. It is conceivable that this could be done by caclculating the correlation scores for 'region' and 'region_code' and 'district code'. Whether or not this could lead to any concrete inights is something to be considered later. 

The attribute 'population' is relevant as it gives us an indication of how many people the waterpoint is servicing. The vast majority of water points have a very small population value. The average is ~ 180. This value is however likely skewed on account of the presence of outliers.  

The attribute 'construction_year' is also important. Unfortuantely, our data here is likely corrupt, as many of our data entries have the value 0 for this variable. As such, our statistics and histogramm above are also likely inaccurate. We will need to consider this in more detail in the sequel. 

In [ ]:
numeric_pump = pump_train.select_dtypes(include=[np.number])

corr = numeric_pump._get_numeric_data().corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, center=0.0,
                      vmax = 1, square=True, linewidths=.5, ax=ax)
plt.show()

Of interest in the above correlation matrix are the scores for 'construction_year'. We see that there is a strong relation between 'gps_height', 'population' and 'longitude'. Any number of hypotheses could be built on top of these relaiton scores. We  will save that discussion for later.  

The fact that 'region_code' correlates with 'district_code' is both unsurprising and irrelevant. 

### Amount of Water

In [ ]:
empty_wp = len(pump_train[(pump_train.amount_tsh > 0)])
print('There are ' + str(empty_wp) + ' non-empty water points in our data set.')
print('This number reflects ' + str(empty_wp / len(pump_train)) + '% of the water points in our data set.') 

This number is seemingly low. Roughly 70% of our data set is composed of water points that don't actually have any water in them. This also raises an important question relating to the data dictionary. 

In [ ]:
functional_wp = len(pump_train[(pump_train.status_group == 'functional')])
functional_np_wp = len(pump_train[(pump_train.status_group == 'functional needs repair')])
print(functional_wp + functional_np_wp)

We have 31113 functional or semi-functional water points in our data, but we have only 15060 non-empty waterpoints. If our data is not corrupt, this means that a functional water point is not necessarily a water point that can actually deliver water. Presumably, a functional water point is thus defined as a water point that is in mechanical order, but which may or may not have any water in it. This is important to keep in mind as we move forward.  

In [ ]:
g = sns.FacetGrid(pump_train, hue='status_group', height=6)
g.map(sns.histplot,'quantity_group',edgecolor="w").add_legend()

Furthermore, one can see that the feature quantity_group "dry" only contains about 5000 wells, which are dry. This does not correspond to the number of wells with no water. This means that most of the values ​​that correspond to 0 stand for missing values.

### Population

In [ ]:
pump_train['population'].loc[pump_train['population']].hist()

The above histogramm isn't particularly helpful. Let's try again with a box plot. 

In [ ]:
pop_bp = sns.boxplot(x=pump_train["population"])

Although the boxplot above is difficult to read, we can see still make out two possible problems in the 'population' attribute. 
1. The presence of a large number of outliers. This is problematic as some machine learning models have difficulties with outliers.  
2. Data entries with a population of 0. This is problematic, as it could indicate some form of data corruption. 

In [ ]:
pop_0 = pump_train.loc[(pump_train.population == 0)]
print(str(len(pop_0)) + ' water points have a population of 0.') 

In [ ]:
pop_1 = pump_train.loc[(pump_train.population == 1)]
print(str(len(pop_1)) + ' water points have a population of 1.') 

In [ ]:
print(str(len(pop_0) + len(pop_1)) + ' water points have a population of either 0 or 1. That is approximately half of our data set.')

Let us see what our data looks like if we remove data entries with a population of 0 or 1:

In [ ]:
pop_2 = pump_train.loc[(pump_train.population > 1)] 
pop_list_2 = pop_2.population.values.tolist()
print('The number of data entries we have after removing 0s and 1s is: ' + str(len(pop_list_2)))
print('The mode of population after removing 0s and 1s is: ' + str(statistics.mode(pop_list_2)))

In [ ]:
pop_2.describe()

In [ ]:
# Now let us remove data entries whose population score is 3 standard deviations above the mean, where the mean and standard
# deviation are calculated after removing 0s and 1s. 

pop_3 = pump_train.loc[(pump_train.population > 1) & (pump_train.population < 2060)] 
len(pop_3)

In [ ]:
pop_bp_3 = sns.boxplot(x=pop_3["population"])

To be very sure, the data dictionary does not provide us with enough information to properly interpret the attribute 'population'. Presumably, it refers to either a) the number of people living within an x metre radius of the water point, or else b) the number of people who the water point regularly services. Without any clarity as to this attribute, it is difficult to know whether the above removal of 0s and 1s was justifiable. We will return to this problem in the data preparation phase. 

### Construction Year

In [ ]:
cy_0 = pump_train['construction_year'].loc[pump_train['construction_year'] == 0]
print('Our data set contains ' + str(len(cy_0)) + ' data entries with a 0 value for construction year.')

After we remove 0s from our data set we get the following data set. 

In [ ]:
pump_train['construction_year'].loc[pump_train['construction_year'] != 0].hist()

The vast majority of waterpoints were built in the last 30 years, with the oldest water points extending back to the 1960's. 

### GPS Height

In [ ]:
pump_train['gps_height'].hist()

In [ ]:
Image(filename='topografic_map.png')

Based on this representation you can see that apart from places by the sea there are no places in Tanzania that are below 100m above sea level. Thus, the values ​​0 in the GPS altitude do not count and can be classified as missing values.

## Categorical Features

In [ ]:
collected_cols = []
for col in pump_train.columns:
    
    if pump_train[col].nunique() < 20:
        collected_cols.append(col)

        

a = len(collected_cols)
b = 2
c = 1  

fig = plt.figure(figsize=(12,len(collected_cols)*6))

for i, col in enumerate(collected_cols):
    plt.subplot(a, b, c)
    plt.title(col)
    plt.xlabel(i)
    pump_train[col].value_counts().plot(kind='bar')
    c = c + 1

plt.tight_layout()
plt.show()
        


There are several insights to be gained from the above graphs. 

1. Many of the attributes are replicated. There is, for example, a pair of attributes called 'payment' and 'payment_type'. In the graphs above we see that they have exactly the same values as one another. In other instances, for example, the pair of attributes 'water_quality' and 'quality_group' the graphs are identical except that 'water_quality' has one additional category with what appears to be no or else extremely few values. In still other instances, for example, 'source' and 'source_type', there is enough of differnce for that difference to be relevant. In the data preparation phase we will need to take this into consideration. 

2. All of the data was recorded by the same company, namely, GeoData Consultants Ltd. We can accordingly drop this attribute as it provides us with no means of differentiating our data. 

3. There appears to be a discrepancy between the numerical attribute 'amount_tsh' (i.e., amount of water) and the categorical attributes 'quantity' and 'quantity_group'. We will need to look at this relationship more closely in the sequel. 

## Target feature

We will start by calling up totals for each of the status_group variables. Then we will visualise them as percentages. 

In [ ]:
print(pump_train['status_group'].value_counts()) 

pump_train['status_group'].value_counts(normalize=True).plot(kind="bar", alpha = 0.5)
plt.title('Water point status totals in %')

Just over half of our water points are fully functional, the rest are either non-functional or else require repairs. 

### Correlation with construction year

In [ ]:
g = sns.FacetGrid(pump_train[pump_train['construction_year'] != 0], hue='status_group', height=6)
g.map(sns.histplot,'construction_year',edgecolor="w").add_legend()

In this plot you can see the correlation with the construction year. It can be seen that the number of wells built increases as the years go by. In addition, one can see that the non-functioning wells in the years 1960 to 1990 are relatively higher than the following ones.

### Correlation with amount_tsh

In [ ]:
g = sns.FacetGrid(pump_train[pump_train['amount_tsh'] == 0], hue='status_group', height=5)
g.map(sns.histplot,'amount_tsh',edgecolor="w", bins=1).add_legend()

In [ ]:
g = sns.FacetGrid(pump_train[pump_train['amount_tsh'] != 0], hue='status_group', height=5)
g.map(sns.histplot,'amount_tsh',edgecolor="w", bins=1).add_legend()

Two plots related to the target variable are presented here. It turns out that when the value is 0, the functionality is higher, but the majority of these fountains still work. For all other non-zero values, you can see that most of these work. So this could be a deciding factor for the models.

### Correlation with gps_height

In [ ]:
g = sns.FacetGrid(pump_train[pump_train['gps_height'] != 0], hue='status_group', height=6)
g.map(sns.histplot,'gps_height',edgecolor="w").add_legend()

In the following plot, the gps height is shown correlated with the target variable. The values ​​0 are excluded for the time being. Here you can see that the functionality is slightly reduced for smaller numbers.

### Correlation with other features

In [ ]:
collected_cols = []
for col in pump_train.columns:
    
    if (pump_train[col].nunique() < 20) and (pump_train[col].nunique() != 2) and col != 'status_group':
        collected_cols.append(col)
print(collected_cols)
        
    
sns.set(font_scale=0.8)

for i, col in enumerate(collected_cols):
    plt.figure(i)
    g = sns.FacetGrid(pump_train, hue='status_group', height=7)
    g.map(sns.histplot,col,edgecolor="w").add_legend()
    

    
 

In the plots shown here, further features are correlated with the target variable. Many connections can be discovered that could later be important for the models

---

In order to get a geographical overview of the data, the individual wells are shown on a map in the following illustrations:

In [ ]:
fig = px.scatter_mapbox(pump_train, lat="latitude", lon="longitude", hover_name="id", hover_data=["waterpoint_type", "gps_height", "status_group"], zoom=3, height=400, color='status_group')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.scatter_mapbox(pump_train, lat="latitude", lon="longitude", hover_name="id", hover_data=["waterpoint_type", "gps_height", "status_group"], zoom=3, height=400, color='status_group')
fig.update_layout(
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ],
    mapbox_style="white-bg"
    
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

On the one hand, these maps show that the wells can be found in Tanzania. You can also interact with the map. The fountains differ in color depending on the functional status they have. When hovering over the individual fountains, additional data can also be seen.

# Data Preparation

### Cleaning Data


As discussed in the analysis, some numeric values ​​were impure because they contained 0 values, which should be considered as missing values. In order to clean up these features, all 0 values ​​for the feature construction_year and gps_height were replaced by NaN, which should be filled in again later. The amount_tsh feature was done in a similar way. However, only the 0 values ​​that do not have the dry attribute in the qunatity_group column were replaced here. This means that probably dry wells should no longer hold water and should remain at 0.

In [ ]:
pump_train['construction_year'] = pump_train['construction_year'].replace(0, np.NaN)
pump_test['construction_year'] = pump_test['construction_year'].replace(0, np.NaN)

pump_train.loc[(pump_train.quantity_group != 'dry') & (pump_train.amount_tsh == 0), 'amount_tsh'] = pump_train.loc[(pump_train.quantity_group != 'dry') & (pump_train.amount_tsh == 0), 'amount_tsh'].replace(0, np.NaN)
pump_test.loc[(pump_test.quantity_group != 'dry') & (pump_test.amount_tsh == 0), 'amount_tsh'] = pump_test.loc[(pump_test.quantity_group != 'dry') & (pump_test.amount_tsh == 0), 'amount_tsh'] .replace(0, np.NaN)

pump_train['gps_height'] = pump_train['gps_height'].replace(0, np.NaN)
pump_test['gps_height'] = pump_test['gps_height'].replace(0, np.NaN)

print(pump_train['construction_year'].isnull().sum())
print(pump_train['gps_height'].isnull().sum())

pump_train.isnull().sum()
pump_test.isnull().sum()

### Preparing Data for Model

In order to prepare the values ​​for the model, the feature groups should first be summarized. First, features are determined which should be removed from the data set, as they do not add any value. After that, all numerical features are summarized and prepared for further processing. In addition, columns are grouped for a custom transform. Finally, the remaining columns for the label encoding and the onehot encoding are split. This is determined using the unique values.

In [ ]:
features_drop = ["id",
                 "scheme_name",
                  "num_private"
                ]

features_scale = pump_train.loc[:,~pump_train.columns.isin(features_drop)].select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns

features_custom = ["public_meeting","permit"]

label_cols = []
onehot_cols = []
for col in pump_train.columns[~pump_train.columns.isin(features_scale) & ~pump_train.columns.isin(features_drop) & ~pump_train.columns.isin(features_custom)]:
    
    if pump_train[col].nunique() <= 5:
        onehot_cols.append(col)
    else:
        label_cols.append(col)


features_label_encode = label_cols
features_onehot_encode = onehot_cols

### Drop features

Here the desired columns are removed in the training data and test data.

In [ ]:
pump_train = pump_train.drop(columns=features_drop)
pump_test = pump_test.drop(columns=features_drop)

### Label Encoding

Here the specified columns are label encoded in the training data and test data.

In [ ]:
labelencoder = LabelEncoder()

pump_train[features_label_encode] = pump_train[features_label_encode].apply(labelencoder.fit_transform)
pump_test[features_label_encode] = pump_test[features_label_encode].apply(labelencoder.fit_transform)

### OneHot Encoding

Here the specified columns are label encoded in the training data and test data.

In [ ]:
one_hot_train = pd.get_dummies(pump_train, columns = features_onehot_encode)
one_hot_test = pd.get_dummies(pump_test, columns = features_onehot_encode)

pump_train = pump_train.drop(features_onehot_encode,axis = 1)
pump_test = pump_test.drop(features_onehot_encode,axis = 1)

pump_train = pump_train.merge(one_hot_train)
pump_test = pump_test.merge(one_hot_test)

### Filling NaN values

Two functions were written to fill in missing values. These enable categorical values ​​as well as numerical values ​​to be predicted by models in order to achieve the best possible data quality.

In [ ]:
def fill_missing_values_cat(df, target_column, drop_columns):
    
    train = df[df[target_column].notnull()]
    test = df[df[target_column].isnull()]
    
    train = train.drop(columns=drop_columns)
    test = test.drop(columns=drop_columns)
    
    x = train.drop(columns=target_column)
    y = train[target_column]
    
    etc = ExtraTreesClassifier(n_jobs=-1)
    etc.fit(x, y)
    y_pred = etc.predict(test.drop(columns=target_column))

    df.loc[(df[target_column].isnull()), target_column ] = y_pred
    
    return df

def fill_missing_values_reg(df, target_column, drop_columns):
    
    train = df[df[target_column].notnull()]
    test = df[df[target_column].isnull()]
    
    train = train.drop(columns=drop_columns)
    test = test.drop(columns=drop_columns)
    
    x = train.drop(columns=target_column)
    y = train[target_column]
    
    rtr = RandomForestRegressor(n_jobs=-1)
    rtr.fit(x, y)
    y_pred = rtr.predict(test.drop(columns=target_column))

    df.loc[(df[target_column].isnull()), target_column ] = y_pred
    
    return df
    

The missing values ​​are now replaced in the following paragraph. For the first three features, only 'Missing' is initially entered as a value, since these contain many names and a prediction here is not productive. For the following features 'construction_year', 'gps_height', 'amount_tsh', 'permit' and 'public_meeting' the values ​​are populated by machine learning models.

In [ ]:
# Funder
pump_train['funder'] = pump_train['funder'].fillna('Missing')
pump_test['funder'] = pump_test['funder'].fillna('Missing')

# Installer
pump_train['installer'] = pump_train['installer'].fillna('Missing')
pump_test['installer'] = pump_test['installer'].fillna('Missing')

# Subvillage
pump_train['subvillage'] = pump_train['subvillage'].fillna('Missing')
pump_test['subvillage'] = pump_test['subvillage'].fillna('Missing')

# scheme_management
pump_train['scheme_management'] = pump_train['scheme_management'].fillna('Missing')
pump_test['scheme_management'] = pump_test['scheme_management'].fillna('Missing')

# Construction year
pump_train = fill_missing_values_reg(pump_train, 'construction_year', ['permit', 'public_meeting','gps_height','amount_tsh'])
pump_test = fill_missing_values_reg(pump_test, 'construction_year', ['permit', 'public_meeting','gps_height','amount_tsh'])

# Amount tsh
pump_train = fill_missing_values_reg(pump_train, 'amount_tsh', ['permit', 'public_meeting','gps_height'])
pump_test = fill_missing_values_reg(pump_test, 'amount_tsh', ['permit', 'public_meeting','gps_height'])

# GPS height
pump_train = fill_missing_values_reg(pump_train, 'gps_height', ['permit', 'public_meeting'])
pump_test = fill_missing_values_reg(pump_test, 'gps_height', ['permit', 'public_meeting'])

# Public meeting (could be predicted)
pump_train['public_meeting'] = pump_train['public_meeting'].replace({True: 1, False: 0})
pump_test['public_meeting'] = pump_test['public_meeting'].replace({True: 1, False: 0})

pump_train = fill_missing_values_cat(pump_train, 'public_meeting', 'permit')
pump_test = fill_missing_values_cat(pump_test, 'public_meeting', 'permit')

# Permit (could be predicted)
pump_train['permit'] = pump_train['permit'].replace({True: 1, False: 0})
pump_test['permit'] = pump_test['permit'].replace({True: 1, False: 0})

pump_train = fill_missing_values_cat(pump_train, 'permit', 'public_meeting')
pump_test = fill_missing_values_cat(pump_test, 'permit', 'public_meeting')



In [ ]:
# pump_train.isnull().sum()
# pump_test.isnull().sum()

### Scaling

In [ ]:
scaler_train = StandardScaler()
scaler_test = StandardScaler()
# transform data
pump_train[features_scale] = scaler_train.fit_transform(pump_train[features_scale])
pump_test[features_scale] = scaler_train.transform(pump_test[features_scale])


# Model: functioning or not

The first model to be created has the purpose of predicting whether the well is functional or not. This is implemented using a wide variety of models and the results are then assessed.


## Model specific adjustements

In [ ]:
target_feature = 'status_group_non functional'

m1_pump_train = pump_train.copy()
m1_pump_train = m1_pump_train.drop(columns=['status_group_functional','status_group_functional needs repair'])
m1_pump_test = pump_test.copy()
m1_pump_test = m1_pump_test.drop(columns=['status_group_functional','status_group_functional needs repair'])


x_train = m1_pump_train.drop(columns=target_feature)
y_train = m1_pump_train[target_feature]
x_val = m1_pump_test.drop(columns=target_feature)
y_val = m1_pump_test[target_feature]


## ML Models

### Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=3000, n_jobs=-1)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict_proba(x_val)

### Gaussian Naive Bias

In [ ]:
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred_gaussian = gaussian.predict_proba(x_val)

### Random Forest Classifier

In [ ]:
randomforest = RandomForestClassifier(n_jobs=-1)
randomforest.fit(x_train, y_train)
y_pred_randomforest = randomforest.predict_proba(x_val)

### XGBoost

In [ ]:
xgb = XGBClassifier(n_jobs=-1)
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict_proba(x_val)

### Extra Trees Classifier

In [ ]:
etc = ExtraTreesClassifier(n_jobs=-1)
etc.fit(x_train, y_train)
y_pred_etc = etc.predict_proba(x_val)

### Hyperparametertuning - Random Forest Classifier

In [ ]:
iterations = 50
folds = 3
cpu_cores = -1

rfc_tuned = RandomForestClassifier(n_jobs=-1)
params = {
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

rfc_tuned_RSCV = RandomizedSearchCV(rfc_tuned, param_distributions=params, random_state=42, n_iter=iterations, cv=folds, verbose=2, n_jobs=cpu_cores, return_train_score=True)
rfc_tuned_RSCV.fit(x_train, y_train)

print(rfc_tuned_RSCV.best_estimator_.get_params())

y_pred_rfc_tuned = rfc_tuned_RSCV.predict_proba(x_val)

### Hyperparametertuning - XGBoost

In [ ]:

# RandomSearchCV XGBoost
iterations = 50
folds = 3
cpu_cores = -1

xgb_tuned = XGBClassifier(n_jobs=-1)
params = {
    "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
    "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight" : [ 1, 3, 5, 7 ],
    "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

xgb_tuned_RSCV = RandomizedSearchCV(xgb_tuned, param_distributions=params, random_state=42, n_iter=iterations, cv=folds, verbose=2, n_jobs=cpu_cores, return_train_score=True)
xgb_tuned_RSCV.fit(x_train, y_train)

print(xgb_tuned_RSCV.best_estimator_.get_params())

y_pred_xgb_tuned = xgb_tuned_RSCV.predict_proba(x_val)

### Voting Classifier

In [ ]:


# Voting Classifier
estimators = []


estimators.append(('randomforest', randomforest))
estimators.append(('xgb', xgb))
estimators.append(('etc', etc))

ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(x_train, y_train)
y_pred_voting = ensemble.predict_proba(x_val)


## Evaluation

In [ ]:
print('Logistic Regression: \n' + str(metrics.roc_auc_score(y_val, y_pred_logreg[:,1])))
print('Gaussian Naive Bias: \n' + str(metrics.roc_auc_score(y_val, y_pred_gaussian[:,1])))
print('Random Forest Classifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_randomforest[:,1])))
print('XGBoost: \n' + str(metrics.roc_auc_score(y_val, y_pred_xgb[:,1])))
print('ExtraTreeClassifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_etc[:,1])))
print('Random Forest Classifier with tuning: \n' + str(metrics.roc_auc_score(y_val, y_pred_rfc_tuned[:,1])))
print('XGBoost with tuning: \n' + str(metrics.roc_auc_score(y_val, y_pred_xgb_tuned[:,1])))
print('VotingClassifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_voting[:,1])))

# Model: Needs repairs

The second model to be created has the purpose of predicting whether the well is needs to be repaired or not. This is also implemented using a wide variety of models and the results are then assessed.

## Model specific adjustements


In [ ]:


target_feature = 'status_group_functional needs repair'

m2_pump_train = pump_train.copy()
m2_pump_train = m2_pump_train.drop(columns=['status_group_functional','status_group_non functional'])
m2_pump_test = pump_test.copy()
m2_pump_test = m2_pump_test.drop(columns=['status_group_functional','status_group_non functional'])


x_train = m2_pump_train.drop(columns=target_feature)
y_train = m2_pump_train[target_feature]
x_val = m2_pump_test.drop(columns=target_feature)
y_val = m2_pump_test[target_feature]


rus = RandomUnderSampler() 

x_train, y_train = rus.fit_resample(x_train, y_train)


## ML Models

### Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=3000, n_jobs=-1)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict_proba(x_val)

### Gaussian Naive Bias

In [ ]:
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred_gaussian = gaussian.predict_proba(x_val)

### Random Forest Classifier

In [ ]:
randomforest = RandomForestClassifier(n_jobs=-1)
randomforest.fit(x_train, y_train)
y_pred_randomforest = randomforest.predict_proba(x_val)

### XGBoost

In [ ]:
xgb = XGBClassifier(n_jobs=-1)
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict_proba(x_val)

### Extra Trees Classifier

In [ ]:
etc = ExtraTreesClassifier(n_jobs=-1)
etc.fit(x_train, y_train)
y_pred_etc = etc.predict_proba(x_val)

### Hyperparametertuning - Random Forest Classifier

In [ ]:
iterations = 50
folds = 3
cpu_cores = -1

rfc_tuned = RandomForestClassifier(n_jobs=-1)
params = {
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

rfc_tuned_RSCV = RandomizedSearchCV(rfc_tuned, param_distributions=params, random_state=42, n_iter=iterations, cv=folds, verbose=2, n_jobs=cpu_cores, return_train_score=True)
rfc_tuned_RSCV.fit(x_train, y_train)

print(rfc_tuned_RSCV.best_estimator_.get_params())

y_pred_rfc_tuned = rfc_tuned_RSCV.predict_proba(x_val)

### Hyperparametertuning - XGBoost

In [ ]:

# RandomSearchCV XGBoost
iterations = 50
folds = 3
cpu_cores = -1

xgb_tuned = XGBClassifier(n_jobs=-1)
params = {
    "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
    "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight" : [ 1, 3, 5, 7 ],
    "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

xgb_tuned_RSCV = RandomizedSearchCV(xgb_tuned, param_distributions=params, random_state=42, n_iter=iterations, cv=folds, verbose=2, n_jobs=cpu_cores, return_train_score=True)
xgb_tuned_RSCV.fit(x_train, y_train)

print(xgb_tuned_RSCV.best_estimator_.get_params())

y_pred_xgb_tuned = xgb_tuned_RSCV.predict_proba(x_val)

### Voting Classifier

In [ ]:


# Voting Classifier
estimators = []


estimators.append(('randomforest', randomforest))
estimators.append(('xgb', xgb))
estimators.append(('etc', etc))

ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(x_train, y_train)
y_pred_voting = ensemble.predict_proba(x_val)

## Evaluation


In [ ]:
print('Logistic Regression: \n' + str(metrics.roc_auc_score(y_val, y_pred_logreg[:,1])))
print('Gaussian Naive Bias: \n' + str(metrics.roc_auc_score(y_val, y_pred_gaussian[:,1])))
print('Random Forest Classifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_randomforest[:,1])))
print('XGBoost: \n' + str(metrics.roc_auc_score(y_val, y_pred_xgb[:,1])))
print('ExtraTreeClassifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_etc[:,1])))
print('Random Forest Classifier with tuning: \n' + str(metrics.roc_auc_score(y_val, y_pred_rfc_tuned[:,1])))
print('XGBoost with tuning: \n' + str(metrics.roc_auc_score(y_val, y_pred_xgb_tuned[:,1])))
print('VotingClassifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_voting[:,1])))

# Model 3: All 3 classes

The third model to be created has the purpose of predicting all three classes the status column offers. This is also implemented using a wide variety of models and the results are then assessed.

## Model specific adjustements

In [ ]:
target_feature = 'status'

m3_pump_train = pump_train.copy()

temp = []
for index, row in m3_pump_train.iterrows():

    if row['status_group_functional'] == 1:
        temp.append(0)
    elif row['status_group_non functional'] == 1:
        temp.append(1)
    elif row['status_group_functional needs repair'] == 1:
        temp.append(2)

m3_pump_train['status'] = temp
m3_pump_train = m3_pump_train.drop(columns=['status_group_functional','status_group_non functional','status_group_functional needs repair'])


m3_pump_test = pump_test.copy()

temp = []
for index, row in m3_pump_test.iterrows():
    if row['status_group_functional'] == 1:
        temp.append(0)
    elif row['status_group_non functional'] == 1:
        temp.append(1)
    elif row['status_group_functional needs repair'] == 1:
        temp.append(2)

m3_pump_test['status'] = temp
m3_pump_test = m3_pump_test.drop(columns=['status_group_functional','status_group_non functional','status_group_functional needs repair'])


x_train = m3_pump_train.drop(columns=target_feature)
y_train = m3_pump_train[target_feature]
x_val = m3_pump_test.drop(columns=target_feature)
y_val = m3_pump_test[target_feature]


## ML Models

### Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=3000, n_jobs=-1)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict_proba(x_val)

### Gaussian Naive Bias

In [ ]:
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred_gaussian = gaussian.predict_proba(x_val)

### Random Forest Classifier

In [ ]:
randomforest = RandomForestClassifier(n_jobs=-1)
randomforest.fit(x_train, y_train)
y_pred_randomforest = randomforest.predict_proba(x_val)

### XGBoost

In [ ]:
xgb = XGBClassifier(n_jobs=-1)
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict_proba(x_val)

### Extra Trees Classifier

In [ ]:
etc = ExtraTreesClassifier(n_jobs=-1)
etc.fit(x_train, y_train)
y_pred_etc = etc.predict_proba(x_val)

### Hyperparametertuning - Random Forest Classifier

In [ ]:
iterations = 50
folds = 3
cpu_cores = -1

rfc_tuned = RandomForestClassifier(n_jobs=-1)
params = {
 'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

rfc_tuned_RSCV = RandomizedSearchCV(rfc_tuned, param_distributions=params, random_state=42, n_iter=iterations, cv=folds, verbose=2, n_jobs=cpu_cores, return_train_score=True)
rfc_tuned_RSCV.fit(x_train, y_train)

print(rfc_tuned_RSCV.best_estimator_.get_params())

y_pred_rfc_tuned = rfc_tuned_RSCV.predict_proba(x_val)

### Hyperparametertuning - XGBoost

In [ ]:

# RandomSearchCV XGBoost
iterations = 50
folds = 3
cpu_cores = -1

xgb_tuned = XGBClassifier(n_jobs=-1)
params = {
    "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
    "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight" : [ 1, 3, 5, 7 ],
    "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

xgb_tuned_RSCV = RandomizedSearchCV(xgb_tuned, param_distributions=params, random_state=42, n_iter=iterations, cv=folds, verbose=2, n_jobs=cpu_cores, return_train_score=True)
xgb_tuned_RSCV.fit(x_train, y_train)

print(xgb_tuned_RSCV.best_estimator_.get_params())

y_pred_xgb_tuned = xgb_tuned_RSCV.predict_proba(x_val)

### Voting Classifier

In [ ]:


# Voting Classifier
estimators = []


estimators.append(('randomforest', randomforest))
estimators.append(('xgb', xgb))
estimators.append(('etc', etc))

ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(x_train, y_train)
y_pred_voting = ensemble.predict_proba(x_val)

## Evaluation


In [ ]:
print('Logistic Regression: \n' + str(metrics.roc_auc_score(y_val, y_pred_logreg, multi_class="ovr", average="weighted")))
print('Gaussian Naive Bias: \n' + str(metrics.roc_auc_score(y_val, y_pred_gaussian, multi_class="ovr", average="weighted")))
print('Random Forest Classifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_randomforest, multi_class="ovr", average="weighted")))
print('XGBoost: \n' + str(metrics.roc_auc_score(y_val, y_pred_xgb, multi_class="ovr", average="weighted")))
print('ExtraTreeClassifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_etc, multi_class="ovr", average="weighted")))
print('Random Forest Classifier with tuning: \n' + str(metrics.roc_auc_score(y_val, y_pred_rfc_tuned, multi_class="ovr", average="weighted")))
print('XGBoost with tuning: \n' + str(metrics.roc_auc_score(y_val, y_pred_xgb_tuned, multi_class="ovr", average="weighted")))
print('VotingClassifier: \n' + str(metrics.roc_auc_score(y_val, y_pred_voting, multi_class="ovr", average="weighted")))